# Importing Libraries


In [1]:
import helper
from helper import *

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
import importlib
importlib.reload(helper)


<module 'helper' from 'c:\\Users\\satya\\OneDrive\\Desktop\\Summer_Internship\\helper.py'>

# Data Collection and cleaning

In [3]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [4]:
symbols = collect_tickers('US', start_date, end_date)

Sucesfully downloaded 835 symbols.


In [5]:
snp_stocks_data = download_data(symbols, start_date, end_date)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

$BF.B: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Could not fetch data for nan: 'float' object has no attribute 'replace'
Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['ATVI']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


...

$DJ: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)
$AV: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['NCR']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['MXIM']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['FDC']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed


Could not fetch data for nan: 'float' object has no attribute 'replace'


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['TSG']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['QTRN']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)')


$QTRN: possibly delisted; No price data found  (1d 2000-01-01 -> 2023-12-31)
Sucesfully downloaded data for 799 stocks!


In [6]:
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)
snp_stocks_data = clean_data(symbols, snp_stocks_data, date_range)

Could not update data for 16 stocks.
Could not update data for 3 stocks.


In [7]:
symbols = list(snp_stocks_data.keys())

In [8]:
returns_df = calculate_return(symbols, snp_stocks_data, end_date)

In [9]:
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_momentum(data)

# Convert the momentum data dictionary to a DataFrame
momentum = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:140: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if monthly_data[i-12] != 0:
c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:141: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum_return = ((monthly_data[i] - monthly_data[i-12]) / monthly_data[i-12]) * 100


In [10]:
# Calculate the 12-1 momentum return for each stock in the snp_stocks_data dictionary
momentum_data = {}
for symbol, data in snp_stocks_data.items():
    momentum_data[symbol] = calculate_12_1_momentum(data)

# Convert the momentum data dictionary to a DataFrame
momentum_12_1 = pd.DataFrame.from_dict(momentum_data, orient='index')

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:159: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if monthly_data[i-12] != 0:
c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:160: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  momentum_return = (monthly_data[i-1] - monthly_data[i-12]) / monthly_data[i-12] * 100


In [11]:
top_p_momentum_stocks_dict = top_p_percentile_stocks(momentum, 10)

In [12]:
top_30_momentum = top_p_percentile_stocks(momentum, 10)

In [13]:
"""
  Selects stocks from dict2 that are not present in dict1.

  Args:
      dict1 (dict): Dictionary with timestamps as keys and lists of timestamps as values.
      dict2 (dict): Dictionary with timestamps as keys and lists of timestamps (stock symbols) as values.

  Returns:
      list: List of stock symbols from dict2 that are not in dict1.
"""
def select_unique_stocks(dict1, dict2):
    unique_stocks_dict = {}

    for timestamp in dict2:
        # Get the list of stocks for the current timestamp from dict2
        stocks_in_dict2 = dict2[timestamp]
        
        # Get the list of stocks for the current timestamp from dict1, if it exists
        stocks_in_dict1 = dict1.get(timestamp, [])
        
        # Select stocks that are in dict2 but not in dict1
        unique_stocks = [stock for stock in stocks_in_dict2 if stock not in stocks_in_dict1]
        
        # Store the unique stocks in the result dictionary
        unique_stocks_dict[timestamp] = unique_stocks
    
    return unique_stocks_dict



In [14]:
top11_30_momentum = select_unique_stocks(top_p_momentum_stocks_dict, top_30_momentum)

In [15]:
top_p_momentum_stocks_dict_12_1 = top_p_percentile_stocks(momentum_12_1, 10)

In [16]:
bottom_p_momentum_stocks_dict = bottom_p_percentile_stocks(momentum, 10)

In [17]:
bottom_p_momentum_stocks_dict_12_1 = bottom_p_percentile_stocks(momentum_12_1, 10)

In [18]:
long_dict = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

In [19]:
long_dict

{Timestamp('2001-01-31 00:00:00'): -5.360261398126364,
 Timestamp('2001-02-28 00:00:00'): -1.4601193283506464,
 Timestamp('2001-03-31 00:00:00'): 5.4624560398687505,
 Timestamp('2001-04-30 00:00:00'): 3.150150856786759,
 Timestamp('2001-05-31 00:00:00'): -1.8659362206021972,
 Timestamp('2001-06-30 00:00:00'): 2.161985157012165,
 Timestamp('2001-07-31 00:00:00'): -3.805598020479907,
 Timestamp('2001-08-31 00:00:00'): -5.854174075764395,
 Timestamp('2001-09-30 00:00:00'): 3.7658551742113704,
 Timestamp('2001-10-31 00:00:00'): 7.826530643787557,
 Timestamp('2001-11-30 00:00:00'): 7.273457359273492,
 Timestamp('2001-12-31 00:00:00'): 2.063798461990609,
 Timestamp('2002-01-31 00:00:00'): 5.332401011706582,
 Timestamp('2002-02-28 00:00:00'): 4.520272211920777,
 Timestamp('2002-03-31 00:00:00'): 4.057119541353977,
 Timestamp('2002-04-30 00:00:00'): -2.0473965898907878,
 Timestamp('2002-05-31 00:00:00'): -4.304098460168974,
 Timestamp('2002-06-30 00:00:00'): -8.179865967719309,
 Timestamp('200

In [20]:
long_dict_12_1 = total_return(snp_stocks_data, top_p_momentum_stocks_dict_12_1, returns_df)

In [21]:
long_dict_inversion = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

In [22]:
sort_dict = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

In [23]:
sort_dict_12_1 = total_return(snp_stocks_data, bottom_p_momentum_stocks_dict_12_1, returns_df)

In [24]:
sort_dict_inversion = total_return(snp_stocks_data, top_p_momentum_stocks_dict, returns_df)

In [25]:
sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)

In [26]:
sort_long_dict_12_1={}
for i in sort_dict_12_1.keys():
    sort_long_dict_12_1[i] = long_dict_12_1[i] -sort_dict_12_1[i]

In [27]:
sort_long_dict_inversion={}
for i in sort_dict_inversion.keys():
    sort_long_dict_inversion[i] = long_dict_inversion[i] -sort_dict_inversion[i]

In [28]:
sort_long_dict

{Timestamp('2001-01-31 00:00:00'): 2.9820069431841945,
 Timestamp('2001-02-28 00:00:00'): 0.21929460894389172,
 Timestamp('2001-03-31 00:00:00'): -6.797164047456317,
 Timestamp('2001-04-30 00:00:00'): -0.12200590497981557,
 Timestamp('2001-05-31 00:00:00'): 0.9149632908227946,
 Timestamp('2001-06-30 00:00:00'): 4.128943225259209,
 Timestamp('2001-07-31 00:00:00'): 9.045645274307718,
 Timestamp('2001-08-31 00:00:00'): 2.6938284190646584,
 Timestamp('2001-09-30 00:00:00'): -0.34648969040367145,
 Timestamp('2001-10-31 00:00:00'): -3.884143582481503,
 Timestamp('2001-11-30 00:00:00'): -2.984080882766058,
 Timestamp('2001-12-31 00:00:00'): 4.971197978045767,
 Timestamp('2002-01-31 00:00:00'): 0.3397027501592485,
 Timestamp('2002-02-28 00:00:00'): -6.345164129537685,
 Timestamp('2002-03-31 00:00:00'): 0.2581500254168176,
 Timestamp('2002-04-30 00:00:00'): 1.030879575704515,
 Timestamp('2002-05-31 00:00:00'): 3.7072750076317558,
 Timestamp('2002-06-30 00:00:00'): 2.1934131125378253,
 Timestam

In [29]:
m = 3

# Call the function
total_returns_dict_ma = total_return_with_moving_average(top_p_momentum_stocks_dict, returns_df, m, snp_stocks_data)


In [30]:

# Apply the trading strategy
trade_returns_seasonality = seasonality_strategy(returns_df, top_p_momentum_stocks_dict)


In [31]:

x = returns_df.loc[['GR', 'NVR', 'ODP', 'TWX', 'BKNG', 'IBM'],'2000-06-30'].abs()
x[x<=0].empty

False

# Benchmark

The S&P500 index is used as a benchmark.

In [32]:
data = benchmark_data('US', start_date, end_date)

[*********************100%%**********************]  1 of 1 completed


In [33]:
gains = benchmark_returns(data)
gains.index = pd.to_datetime(gains.index, format='%Y-%m-%d')

In [34]:
# Define a function to calculate 12-month momentum for S&P 500
def calculate_12_month_momentum(data):
    return data.pct_change(12).dropna()

benchmark = data.loc[:'2024-05-31 00:00:00']
sp500_momentum = calculate_12_month_momentum(benchmark)




# Function to compare stock momentum with S&P 500 and calculate total returns
def compare_and_calculate_returns(top_p_momentum_stocks_dict, momentum_df, returns_df, sp500_momentum, sp500_returns, max_val=50):
    # Initialize a dictionary to store the adjusted momentum returns
    adjusted_returns_dict = {}

    for timestamp, stocks in top_p_momentum_stocks_dict.items():
        stock_returns = []
        for stock in stocks:            
            if sp500_momentum.loc[timestamp] > momentum_df.loc[stock, timestamp]:
                stock_returns.append(sp500_returns.loc[timestamp])
            elif abs(returns_df.loc[stock, timestamp]) > max_val:
                stock_returns.append(0)
            else:
                stock_returns.append(returns_df.loc[stock, timestamp])
        
        total_return = sum(stock_returns)/len(stock_returns)

        adjusted_returns_dict[timestamp] = total_return

    # Convert the adjusted returns dictionary to a DataFrame
    adjusted_returns_series = pd.Series(adjusted_returns_dict)
    adjusted_returns_series.index = pd.to_datetime(adjusted_returns_series.index)

    return adjusted_returns_series

# Example usage
adjusted_returns_series = compare_and_calculate_returns(top_p_momentum_stocks_dict, momentum, returns_df, sp500_momentum, gains)


KeyError: Timestamp('2001-03-31 00:00:00')

In [ ]:

results_df = calculate_yearly_stats(sort_long_dict, gains)


c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:353: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

c:\Users\satya\OneDrive\Desktop\Summer_Internship\helper.py:365: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



# Plots

## 12-months momentum and 1-month return

### Going long only for the top 10% stocks

In [35]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [37]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [38]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [39]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns,
        mode='lines',
        name='Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [40]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T.round(2)

2001-12-31    19.201174
2002-12-31    -3.889280
2003-12-31    23.996038
2004-12-31    10.986365
2005-12-31    16.921550
2006-12-31    -1.787966
2007-12-31    -7.202791
2008-12-31   -34.160274
2009-12-31     0.560103
2010-12-31   -11.884383
2011-12-31    -1.345309
2012-12-31     5.240870
2013-12-31    11.386736
2014-12-31   -11.079817
2015-12-31   -24.320369
2016-12-31   -10.444487
2017-12-31     9.743605
2018-12-31   -20.393574
2019-12-31     8.235631
2020-12-31     0.919668
2021-12-31    -2.359643
2022-12-31    -3.185262
2023-12-31    12.353792
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,19.20,-3.89,24.00,10.99,16.92,-1.79,-7.20,-34.16,0.56,-11.88,...,-11.08,-24.32,-10.44,9.74,-20.39,8.24,0.92,-2.36,-3.19,12.35
Benchmark Returns,-7.41,-23.37,26.38,8.99,3.00,13.62,3.53,-38.49,23.45,12.78,...,11.39,-0.73,9.54,19.42,-6.24,28.88,16.26,26.89,-19.44,24.23
Strategy Sharpe Ratio,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,...,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04,-0.04


### Going long on top 10% and short on bottom 10%

In [41]:
# Convert the total_returns_dict to a DataFrame
total_returns_df = pd.DataFrame.from_dict(sort_long_dict, orient='index', columns=['Total Returns'])
total_returns_df.sort_index(inplace=True)


In [60]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df.index,
        y=total_returns_df['Total Returns'],
        name='Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='Momentum Return(long and sort) vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [43]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns = (1 + total_returns_df['Total Returns'] / 100).cumprod() - 1


In [61]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns.index,
        y=cumulative_momentum_returns,
        mode='lines',
        name='Cumulative Momentum Returns(long-sort)'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Momentum(long-sort) vs Benchmark from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [45]:
results_df = calculate_yearly_stats(sort_long_dict, gains)
results_df.T

2001-12-31    10.278824
2002-12-31     2.975098
2003-12-31    -6.770612
2004-12-31     1.421324
2005-12-31    -4.445226
2006-12-31    -5.120422
2007-12-31     7.015728
2008-12-31    10.349053
2009-12-31   -15.857794
2010-12-31   -18.713832
2011-12-31    22.172438
2012-12-31    -5.738791
2013-12-31    -2.192506
2014-12-31     0.172464
2015-12-31    15.468254
2016-12-31     4.662091
2017-12-31    -6.491000
2018-12-31    -3.022788
2019-12-31    -6.515538
2020-12-31   -19.951593
2021-12-31     3.390019
2022-12-31     4.734543
2023-12-31    -0.305643
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,10.278824,2.975098,-6.770612,1.421324,-4.445226,-5.120422,7.015728,10.349053,-15.857794,-18.713832,...,0.172464,15.468254,4.662091,-6.491000,-3.022788,-6.515538,-19.951593,3.390019,4.734543,-0.305643
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,11.390634,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501
Strategy Sharpe Ratio,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,...,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375,-0.053375


### 12-1 momentum strategy

In [46]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_12_1 = pd.DataFrame.from_dict(long_dict_12_1, orient='index', columns=['Total Returns'])
total_returns_df_12_1.sort_index(inplace=True)


In [47]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_12_1.index,
        y=total_returns_df_12_1['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title='12-1 Momentum Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [48]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_12_1 = gains.reindex(total_returns_df_12_1.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_12_1 = (1 + aligned_benchmark_gains_12_1 / 100).cumprod() - 1
cumulative_momentum_returns_12_1 = (1 + total_returns_df_12_1['Total Returns'] / 100).cumprod() - 1


In [49]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_12_1.index,
        y=cumulative_benchmark_returns_12_1,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_12_1.index,
        y=cumulative_momentum_returns_12_1,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [50]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T

2001-12-31    19.201174
2002-12-31    -3.889280
2003-12-31    23.996038
2004-12-31    10.986365
2005-12-31    16.921550
2006-12-31    -1.787966
2007-12-31    -7.202791
2008-12-31   -34.160274
2009-12-31     0.560103
2010-12-31   -11.884383
2011-12-31    -1.345309
2012-12-31     5.240870
2013-12-31    11.386736
2014-12-31   -11.079817
2015-12-31   -24.320369
2016-12-31   -10.444487
2017-12-31     9.743605
2018-12-31   -20.393574
2019-12-31     8.235631
2020-12-31     0.919668
2021-12-31    -2.359643
2022-12-31    -3.185262
2023-12-31    12.353792
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,19.201174,-3.889280,23.996038,10.986365,16.921550,-1.787966,-7.202791,-34.160274,0.560103,-11.884383,...,-11.079817,-24.320369,-10.444487,9.743605,-20.393574,8.235631,0.919668,-2.359643,-3.185262,12.353792
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,11.390634,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501
Strategy Sharpe Ratio,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,...,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221,-0.038221


### Buying only if the current price is above the 6-month MA

In [51]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_ma = pd.DataFrame.from_dict(total_returns_dict_ma, orient='index', columns=['Total Returns'])
total_returns_df_ma.sort_index(inplace=True)


In [63]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_ma.index,
        y=total_returns_df_ma['Total Returns'],
        name='Strategy Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Price > 6-ma vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [53]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains_ma = gains.reindex(total_returns_df_ma.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_ma = (1 + aligned_benchmark_gains_ma / 100).cumprod() - 1
cumulative_momentum_returns_ma = (1 + total_returns_df_ma['Total Returns'] / 100).cumprod() - 1


In [64]:
# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_ma.index,
        y=cumulative_momentum_returns_ma,
        mode='lines',
        name='Cumulative Strategy Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Price >6-ma vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [55]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(total_returns_dict_ma, gains)

yearly_stats_df.T

2001-12-31     1.829883
2002-12-31   -12.267532
2003-12-31    65.005464
2004-12-31    23.474879
2005-12-31    46.642639
2006-12-31    11.034631
2007-12-31    12.533605
2008-12-31   -37.541553
2009-12-31    28.990254
2010-12-31    34.906482
2011-12-31    19.784571
2012-12-31    23.952738
2013-12-31    39.488085
2014-12-31     5.827441
2015-12-31    -6.341867
2016-12-31    -0.616828
2017-12-31     9.631401
2018-12-31    -7.364248
2019-12-31    20.359065
2020-12-31    78.586147
2021-12-31    22.907667
2022-12-31     1.700601
2023-12-31    18.117411
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,1.829883,-12.267532,65.005464,23.474879,46.642639,11.034631,12.533605,-37.541553,28.990254,34.906482,...,5.827441,-6.341867,-0.616828,9.631401,-7.364248,20.359065,78.586147,22.907667,1.700601,18.117411
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,11.390634,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501
Strategy Sharpe Ratio,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,...,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880,0.688880


#### Buy stocks if momentum > S&P500

## Inversion Strategy

#### long the bottom 10%

In [ ]:
# Convert the total_returns_dict to a DataFrame
total_returns_df_inversion = pd.DataFrame.from_dict(long_dict_inversion, orient='index', columns=['Total Returns'])
total_returns_df_inversion.sort_index(inplace=True)


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=total_returns_df_inversion.index,
        y=total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [ ]:
# Ensure both DataFrames have the same index for correct comparison
aligned_benchmark_gains = gains.reindex(total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns_inversions = (1 + aligned_benchmark_gains / 100).cumprod() - 1
cumulative_momentum_returns_inversions = (1 + total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [ ]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_ma.index,
        y=cumulative_benchmark_returns_ma,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_momentum_returns_inversions.index,
        y=cumulative_momentum_returns_inversions,
        mode='lines',
        name='12-1 Cumulative Momentum Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: 12-1 Momentum vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [ ]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(long_dict_inversion, gains)

yearly_stats_df.T

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:17: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,-8.339661,-28.621838,64.139000,13.743506,23.023560,28.071439,-15.053489,-48.238574,80.887441,39.900061,...,-17.730483,26.741933,10.179376,-1.730873,10.380575,29.251004,31.383599,-1.820578,5.930976,2.532949
Benchmark Returns,-15.953767,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,...,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366,0.435366
Benchmark Sharpe Ratio,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,...,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469


#### Long on bottom 10% and short on top 10%

In [ ]:
# Convert the total_returns_dict to a DataFrame
s_l_total_returns_df_inversion = pd.DataFrame.from_dict(sort_long_dict_inversion, orient='index', columns=['Total Returns'])
s_l_total_returns_df_inversion.sort_index(inplace=True)


In [ ]:
# Create a Plotly figure
fig = go.Figure()

# Add the benchmark returns bar trace
fig.add_trace(
    go.Bar(
        x=gains.index,
        y=gains.values,
        name='Benchmark Returns'
    )
)

# Add the momentum returns bar trace
fig.add_trace(
    go.Bar(
        x=s_l_total_returns_df_inversion.index,
        y=s_l_total_returns_df_inversion['Total Returns'],
        name='12-1 Momentum Returns'
    )
)

# Update the layout to include a range slider
fig.update_layout(
    title=' Inversion Return(sort and long) vs Benchmark Return from 01-01-2000',
    xaxis_title='Date',
    yaxis_title='Return %',
    barmode='group',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# Show the plot
fig.show()

In [ ]:
# Ensure both DataFrames have the same index for correct comparison
s_l_aligned_benchmark_gains = gains.reindex(s_l_total_returns_df_inversion.index).fillna(0)

# Calculate cumulative returns
s_l_cumulative_benchmark_returns_inversions = (1 + s_l_aligned_benchmark_gains / 100).cumprod() - 1
s_l_cumulative_momentum_returns_inversions = (1 + s_l_total_returns_df_inversion['Total Returns'] / 100).cumprod() - 1


In [ ]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns_inversions.index,
        y=cumulative_benchmark_returns_inversions,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=s_l_cumulative_momentum_returns_inversions.index,
        y=s_l_cumulative_momentum_returns_inversions,
        mode='lines',
        name='Momentum inversion Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Inversion vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [ ]:
# Calculate the yearly stats
yearly_stats_df = calculate_yearly_stats(sort_long_dict_inversion, gains)

yearly_stats_df.T

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:5: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

C:\Users\satya\AppData\Local\Temp\ipykernel_25332\3420669744.py:17: FutureWarning:

'Y' is deprecated and will be removed in a future version, please use 'YE' instead.



Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Strategy Returns,-9.613155,-8.778563,19.216087,4.995825,-8.615698,9.805042,-23.200976,-6.136139,37.104537,7.822143,...,-14.065914,14.457479,-19.106177,-1.846799,-5.161210,6.833471,21.271645,-11.482898,-12.564045,-5.102908
Benchmark Returns,-15.953767,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501,5.573781
Strategy Sharpe Ratio,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,...,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547,-0.048547
Benchmark Sharpe Ratio,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,...,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469,0.405469


## Seasonality

#### Buy from November to May

In [56]:
# Ensure both DataFrames have the same index for correct comparison
strategy_gains = trade_returns_seasonality.reindex(gains.index).fillna(0)

# Calculate cumulative returns
cumulative_benchmark_returns = (1 + gains / 100).cumprod() - 1
seasonality_cumulative_momentum_returns = (1 + strategy_gains['Trade Returns'] / 100).cumprod() - 1


In [65]:

# Create a Plotly figure
fig = go.Figure()

# Add the cumulative benchmark returns line trace
fig.add_trace(
    go.Scatter(
        x=cumulative_benchmark_returns.index,
        y=cumulative_benchmark_returns,
        mode='lines',
        name='Cumulative Benchmark Returns'
    )
)

# Add the cumulative momentum returns line trace
fig.add_trace(
    go.Scatter(
        x=seasonality_cumulative_momentum_returns.index,
        y=seasonality_cumulative_momentum_returns,
        mode='lines',
        name='Seasonality Returns'
    )
)

# Configure the x-axis to show months when zoomed in and add range selector buttons
fig.update_layout(
    title='Cumulative Returns: Seasonality vs Benchmark from 01-01-2001',
    xaxis_title='Date',
    yaxis_title='Cumulative Return %',
    xaxis=dict(
        tickmode='auto',
        rangeslider=dict(
            visible=True
        ),
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        type="date"
    ),
)

# Show the plot
fig.show()

In [59]:
# Calculate the yearly stats
returns_dict = strategy_gains['Trade Returns'].to_dict()
yearly_stats_df = calculate_yearly_stats(returns_dict, gains)

yearly_stats_df.T

2001-12-31     9.033191
2002-12-31    -0.741382
2003-12-31    17.432095
2004-12-31     5.149441
2005-12-31    24.050475
2006-12-31     5.847097
2007-12-31    -2.234683
2008-12-31     9.104511
2009-12-31     1.573621
2010-12-31     7.560530
2011-12-31    14.192406
2012-12-31    17.928871
2013-12-31    10.940385
2014-12-31    10.893697
2015-12-31    -0.592295
2016-12-31    12.778548
2017-12-31     9.450395
2018-12-31     3.828004
2019-12-31    11.879935
2020-12-31     4.354801
2021-12-31     6.146506
2022-12-31     0.687794
2023-12-31     4.229789
Freq: YE-DEC, dtype: float64


Year,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,9.033191,-0.741382,17.432095,5.149441,24.050475,5.847097,-2.234683,9.104511,1.573621,7.560530,...,10.893697,-0.592295,12.778548,9.450395,3.828004,11.879935,4.354801,6.146506,0.687794,4.229789
Benchmark Returns,-7.408422,-23.365964,26.380400,8.993452,3.001023,13.619432,3.529573,-38.485793,23.454190,12.782714,...,11.390634,-0.726600,9.535023,19.419966,-6.237260,28.878070,16.258923,26.892739,-19.442827,24.230501
Strategy Sharpe Ratio,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,...,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615,1.211615


______________________________________________________________________________________________________________________________________________________

In [ ]:

def calculate_sharpe_ratio(monthly_returns, risk_free_rate=0):
    # Calculate the average monthly return
    average_monthly_return = monthly_returns.mean()
    
    # Calculate the standard deviation of the monthly returns
    monthly_std_dev = monthly_returns.std()
    
    # Calculate the annualized return
    annualized_return = (1 + average_monthly_return / 100) ** (12) - 1
    
    # Calculate the annualized standard deviation
    annualized_std_dev = monthly_std_dev * np.sqrt(12)
    
    # Calculate the Sharpe ratio
    sharpe_ratio = (average_monthly_return - risk_free_rate) / annualized_std_dev
    
    return sharpe_ratio, annualized_return * 100  # Return annualized return as a percentage

def calculate_yearly_stats(strategy_dict, benchmark_returns, risk_free_rate=0):
    # Convert the strategy dictionary to a DataFrame
    strategy_returns_df = pd.Series(strategy_dict)
    strategy_returns_df.index = pd.to_datetime(strategy_returns_df.index)
    strategy_returns_df = strategy_returns_df.resample('ME').last()  # Ensure monthly frequency
    
    # Convert the benchmark returns to a DataFrame
    benchmark_returns.index = pd.to_datetime(benchmark_returns.index)
    benchmark_returns = benchmark_returns.resample('ME').last()  # Ensure monthly frequency
    
    # Initialize lists to store yearly results
    years = []
    strategy_returns = []
    benchmark_returns_list = []
    strategy_sharpe_ratios = []
    benchmark_sharpe_ratios = []

    # Calculate stats for each year
    for year in strategy_returns_df.index.year.unique():
        # Filter the returns for the current year
        strategy_yearly_returns = strategy_returns_df[strategy_returns_df.index.year == year]
        benchmark_yearly_returns = benchmark_returns[benchmark_returns.index.year == year]

        # Ensure we have 12 months of data
        if len(strategy_yearly_returns) == 12 and len(benchmark_yearly_returns) == 12:
        
            # Calculate Sharpe ratio
            strategy_sharpe_ratio, strategy_annual_return = calculate_sharpe_ratio(strategy_yearly_returns, risk_free_rate)
            benchmark_sharpe_ratio, benchmark_annual_return = calculate_sharpe_ratio(benchmark_yearly_returns, risk_free_rate)

            # Append results
            years.append(year)
            strategy_returns.append(strategy_annual_return)
            benchmark_returns_list.append(benchmark_annual_return)
            strategy_sharpe_ratios.append(strategy_sharpe_ratio)
            benchmark_sharpe_ratios.append(benchmark_sharpe_ratio)

    # Create a DataFrame to store the results
    results_df = pd.DataFrame({
        'Year': years,
        'Strategy Returns': strategy_returns,
        'Benchmark Returns': benchmark_returns_list,
        'Strategy Sharpe Ratio': strategy_sharpe_ratios,
        'Benchmark Sharpe Ratio': benchmark_sharpe_ratios
    })

    results_df.set_index('Year', inplace=True)    
    
    return results_df

In [ ]:
yearly_stats_df = calculate_yearly_stats(long_dict_12_1, gains)

yearly_stats_df.T.round(2)

Year,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Strategy Returns,-2.70,29.55,18.24,27.61,3.77,-2.96,-43.65,20.09,-19.05,-14.85,...,-6.32,-17.31,-10.17,19.94,-43.16,8.45,23.31,9.64,9.89,25.91
Benchmark Returns,-21.81,27.10,9.25,3.29,13.79,3.97,-37.08,26.30,14.71,1.15,...,11.72,0.11,10.05,19.50,-5.19,29.85,19.87,27.59,-17.44,25.42
Strategy Sharpe Ratio,-0.01,0.16,0.09,0.10,0.02,-0.01,-0.12,0.09,-0.07,-0.04,...,-0.02,-0.06,-0.04,0.08,-0.18,0.06,0.06,0.02,0.07,0.10
Benchmark Sharpe Ratio,-0.10,0.18,0.10,0.03,0.19,0.03,-0.18,0.09,0.06,0.01,...,0.11,0.00,0.08,0.38,-0.03,0.17,0.06,0.19,-0.07,0.13


In [ ]:
results_df = yearly_stats_df
# Plot the Sharpe ratios
fig = go.Figure()
    
    # Add strategy Sharpe ratios bar trace
fig.add_trace(
    go.Bar(
        x=results_df.index,
        y=results_df['Strategy Sharpe Ratio'],
        name='Strategy Sharpe Ratio'
    )
)
    
    # Add benchmark Sharpe ratios bar trace
fig.add_trace(
    go.Bar(
        x=results_df.index,
        y=results_df['Benchmark Sharpe Ratio'],
        name='Benchmark Sharpe Ratio'
    )
)
    
    # Update the layout
fig.update_layout(
    title='Yearly Strategy vs Benchmark Sharpe Ratios',
    xaxis_title='Year',
    yaxis_title='Sharpe Ratio',
    barmode='group'
)
    
fig.show()

In [ ]:
long_dict

{Timestamp('2001-01-31 00:00:00'): -5.360236191011982,
 Timestamp('2001-02-28 00:00:00'): -1.4601102792206038,
 Timestamp('2001-03-31 00:00:00'): 5.462438394598306,
 Timestamp('2001-04-30 00:00:00'): 3.150151662808117,
 Timestamp('2001-05-31 00:00:00'): -1.865647362381352,
 Timestamp('2001-06-30 00:00:00'): 2.162487307787101,
 Timestamp('2001-07-31 00:00:00'): -3.805081603766806,
 Timestamp('2001-08-31 00:00:00'): -5.855143373664438,
 Timestamp('2001-09-30 00:00:00'): 3.765097253080987,
 Timestamp('2001-10-31 00:00:00'): 7.825469679754791,
 Timestamp('2001-11-30 00:00:00'): 7.274338208460712,
 Timestamp('2001-12-31 00:00:00'): 2.062747393448474,
 Timestamp('2002-01-31 00:00:00'): 5.332818083302102,
 Timestamp('2002-02-28 00:00:00'): 4.517858880268532,
 Timestamp('2002-03-31 00:00:00'): 4.056254831114115,
 Timestamp('2002-04-30 00:00:00'): -2.0461324150200086,
 Timestamp('2002-05-31 00:00:00'): -4.304542781687773,
 Timestamp('2002-06-30 00:00:00'): -8.179458790841373,
 Timestamp('2002-0

In [ ]:
sort_long_dict = long_sort_return(snp_stocks_data, top_p_momentum_stocks_dict, bottom_p_momentum_stocks_dict, returns_df)

In [ ]:
total_return(snp_stocks_data, bottom_p_momentum_stocks_dict, returns_df)

{Timestamp('2001-01-31 00:00:00'): -11.510414729898818,
 Timestamp('2001-02-28 00:00:00'): -0.398165117415966,
 Timestamp('2001-03-31 00:00:00'): 12.572787247712508,
 Timestamp('2001-04-30 00:00:00'): 0.25650192165149954,
 Timestamp('2001-05-31 00:00:00'): -1.923829588527738,
 Timestamp('2001-06-30 00:00:00'): -5.850572296121416,
 Timestamp('2001-07-31 00:00:00'): -16.44960003058977,
 Timestamp('2001-08-31 00:00:00'): -4.4144535120076265,
 Timestamp('2001-09-30 00:00:00'): 0.6528550016546848,
 Timestamp('2001-10-31 00:00:00'): 7.183458393504679,
 Timestamp('2001-11-30 00:00:00'): 4.842498070753337,
 Timestamp('2001-12-31 00:00:00'): -7.08933112877585,
 Timestamp('2002-01-31 00:00:00'): -0.5798539494056716,
 Timestamp('2002-02-28 00:00:00'): 11.512150800954087,
 Timestamp('2002-03-31 00:00:00'): -0.4369759692068638,
 Timestamp('2002-04-30 00:00:00'): -3.4275635146292296,
 Timestamp('2002-05-31 00:00:00'): -9.421832958482009,
 Timestamp('2002-06-30 00:00:00'): -5.335706688418725,
 Timest

In [ ]:
mont_r = d.resample('ME').last()

In [ ]:
x = pd.DataFrame(mont_r.pct_change(-1).loc['2001-01-31':'2024-04-30'])

In [ ]:
x

,Adj Close
Date,
2001-01-31,-0.065673
2001-02-28,0.106539
2001-03-31,0.083221
2001-04-30,-0.095204
2001-05-31,0.004577
...,...
2023-12-31,-0.031247
2024-01-31,-0.033297
2024-02-29,-0.044946


In [ ]:
(1 + x['Adj Close']).cumprod()-1

Date
2001-01-31   -0.065673
2001-02-28    0.033870
2001-03-31    0.119909
2001-04-30    0.013289
2001-05-31    0.017927
                ...   
2023-12-31   -0.729429
2024-01-31   -0.738439
2024-02-29   -0.750195
2024-03-31   -0.755277
2024-04-30   -0.744287
Freq: ME, Name: Adj Close, Length: 280, dtype: float64

In [ ]:
mont_r

Date
2000-01-31    1394.459961
2000-02-29    1366.420044
2000-03-31    1498.579956
2000-04-30    1452.430054
2000-05-31    1420.599976
                 ...     
2024-02-29    5096.270020
2024-03-31    5254.350098
2024-04-30    5035.689941
2024-05-31    5277.509766
2024-06-30    5283.399902
Freq: ME, Name: Adj Close, Length: 294, dtype: float64

In [ ]:
mont_r.shift(-1)

Date
2000-01-31    1366.420044
2000-02-29    1498.579956
2000-03-31    1452.430054
2000-04-30    1420.599976
2000-05-31    1454.599976
                 ...     
2024-02-29    5254.350098
2024-03-31    5035.689941
2024-04-30    5277.509766
2024-05-31    5283.399902
2024-06-30            NaN
Freq: ME, Name: Adj Close, Length: 294, dtype: float64